[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/Joshbazz/Neural_Network_Trading_Algo/main/main.ipynb)

In [5]:
from neural_network_trading_algo.modeling.dnn_live import StockPricePrediction
from sklearn.model_selection import train_test_split
from neural_network_trading_algo.modeling.generate_signals_live import generate_signal
# import model_plot
from neural_network_trading_algo.backtester.backtester_live import *
from neural_network_trading_algo.data.fetch_data_test import fetch_and_save_data


# Example usage
# Define the data path
ticker = '^STI'  # Example ticker
start_date = '2010-01-01'
end_date = '2017-01-03'
file_path = f'{ticker}_data_end_{end_date}_start_{start_date}.csv'


fetch_and_save_data(ticker, start_date, end_date, file_path)


# data_string = 'data/old_ohlcv/BTC-6h-2023-01-01-to-2024-06-27.csv'

# Create an instance of the class
stock_predictor = StockPricePrediction(file_path)

# set the test_size. if test size is .25, then 75% of data will be used to train
test_size = 0.25

# Load and preprocess data
# train_percent [0,1] => determines percent of data that goes to training
data, data_create = stock_predictor.load_and_preprocess_data()

# Create dataset
dataset = stock_predictor.create_dataset(data_create)
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

# retain X_test slice of original dataframe
backtest_data = data.iloc[-len(X_test):].copy()

# Normalize data
X_train_scaled, y_train_scaled = stock_predictor.normalize_data(X_train, y_train)
X_test_scaled, y_test_scaled = stock_predictor.scaler.transform(X_test), stock_predictor.y_scaler.transform(y_test.values.reshape(-1, 1))

# Build and train model
stock_predictor.build_model()
stock_predictor.train_model(X_train_scaled, y_train_scaled, epochs=50)

# # Evaluate model
loss, predictions = stock_predictor.evaluate_model(X_test_scaled, y_test_scaled)

# Inverse transform predictions and actual values
y_test_inv, predictions_inv = stock_predictor.inverse_transform(y_test_scaled, predictions)

# Save the model
model_path = stock_predictor.save_model(epochs=50)

# uncomment when bug fixed
# model_plot.save_and_visualize_model(model_path, img_dir='visualization')

# Generate Signals csv
signals_string = generate_signal(model_path, file_path, test_size)

# Call the function with the path to your CSV file
run_backtest(signals_string)

[*********************100%%**********************]  1 of 1 completed

Data for ^STI from 2010-01-01 to 2017-01-03 has been saved to ^STI_data_end_2017-01-03_start_2010-01-01.csv
Epoch 1/50



/opt/anaconda3/envs/neural_network_trading_algo/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2773 - mean_absolute_percentage_error: 111.2286 - val_loss: 0.2734 - val_mean_absolute_percentage_error: 85.4548
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 0.2464 - mean_absolute_percentage_error: 102.3252 - val_loss: 0.2571 - val_mean_absolute_percentage_error: 82.6906
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 0.2346 - mean_absolute_percentage_error: 98.8165 - val_loss: 0.2446 - val_mean_absolute_percentage_error: 81.5346
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 0.2249 - mean_absolute_percentage_error: 96.5647 - val_loss: 0.2343 - val_mean_absolute_percentage_error: 81.0344
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 0.2286 - mean_absolute_percentage_error: 91.1034 - val_loss: 0.2254 - val_mean_absolute_percentage_error: 80.6859
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.2236 - mean_absolute_percentage_error: 100.4376 - val_loss: 0.2176 - val_mean_a

/opt/anaconda3/envs/neural_network_trading_algo/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/envs/neural_network_trading_algo/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/envs/neural_network_trading_algo/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/opt/anaconda3/envs/neural_network_trading_algo/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the lat